# MOHID Preprocessing

- Create regular grids
- Download, load and filter coastlines for grid region
- Perform interpolation on bathymetric data
- Update griddata depth values
- Convert the griddata to a shapefile
- Plot bathymetry

***
**Note 1**: Execute each cell through the <button class="btn btn-default btn-xs"><i class="icon-play fa fa-play"></i></button> button from the top MENU (or keyboard shortcut `Shift` + `Enter`).<br>
<br>
**Note 2**: Use the Kernel and Cell menus to restart the kernel and clear outputs.<br>
***

# Table of contents
- [1. Import required libraries](#1.-Import-required-librarie)
- [2. Load the XYZ data](#2.-Load-the-XYZ-data)
- [3. Get grid dimensions and spacing](#3.-Get-grid-dimensions-and-spacing)
- [4. Grid generation](#4.-Grid-generation)
- [5. Save the grid to a MOHID-compatible file](#5.-Save-the-grid-to-a-MOHID-compatible-file)
- [6. Download or load GSHHG Coastline Data](#6.-Download-or-load-GSHHG-Coastline-Data)
- [7. Load and Filter Coastlines for Grid Region](#7.-Load-and-Filter-Coastlines-for-Grid-Region)
- [8. Interpolate bathymetric data](#8.-Interpolate-bathymetric-data)
- [9. Load a previously generated Mohid griddata file](#9.-Load-a-previously-generated-Mohid-griddata-file)
- [10. Visualize and update depth values by clicking on the map](#10.-Visualize-and-update-depth-values-by-clicking-on-the-map)
- [11. Save the griddata to a MOHID-compatible file](#11.-Save-the-griddata-to-a-MOHID-compatible-file)
- [12. Convert the griddata to a shapefile](#12.-Convert-the-griddata-to-a-shapefile)
- [13. Save shapefile to MOHID griddata](#13.-Save-shapefile-to-MOHID-griddata)
- [14. Plot interpolated bathymetry](#14.-Plot-interpolated-bathymetry)


# 1. Import required libraries

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import clear_output, display
from ipyleaflet import Map, Marker, basemaps, Popup, Polyline, Circle, GeoData, Polygon, GeoJSON
from ipywidgets import HTML
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.colors as mcolors
from matplotlib.colors import Normalize, to_hex
import random
from scipy.interpolate import griddata
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from urllib.request import urlopen, Request
import io
from PIL import Image
import geopandas as gpd
from shapely.geometry import Point, box
import requests
import zipfile
import os
from scipy.ndimage import label, find_objects, gaussian_filter
import matplotlib.colors as mcolors
import ipywidgets as widgets
import shapefile
from mpl_toolkits.axes_grid1 import make_axes_locatable

# 2. Load the XYZ data
This step is optional (only perform if you already have an xyz file with bathymetry data).

In [ ]:
file_path = 'data.xyz'  # Replace with your file path
data = pd.read_csv(file_path, header=None, names=['longitude', 'latitude', 'depth'], sep='\\s+')
# Convert to list of dictionaries
data_list = data.to_dict(orient='records')

# Define the maximum number of points to display
MAX_POINTS = 1000  # Adjust based on performance needs

# If the dataset is too large, randomly sample points
if len(data_list) > MAX_POINTS:
    data_sample = random.sample(data_list, MAX_POINTS)

# Calculate the mean latitude and longitude for centering the map
center_lat = np.mean([row['latitude'] for row in data_sample])
center_lon = np.mean([row['longitude'] for row in data_sample])
grid_map = Map(center=(center_lat, center_lon), zoom=10)

# Define colormap and normalization
cmap = cm.viridis  # Choose a colormap like 'viridis', 'plasma', etc.
norm = Normalize(vmin=min(row['depth'] for row in data_sample), vmax=max(row['depth'] for row in data_sample))

# Function to convert a value into a color
def value_to_color(value):
    return to_hex(cmap(norm(value)))  # Convert RGBA to hex

# Add circles to the map
for row in data_sample:
    marker = Circle(
        location=(row['latitude'], row['longitude']),
        radius=30,
        color=value_to_color(row['depth']),
        fill_color=value_to_color(row['depth']),
        fill_opacity=0.7
    )
    grid_map.add_layer(marker)

# 3. Get grid dimensions and spacing

In [ ]:
nx = int(input("Enter the number of cells in the x-direction (nx): "))
ny = int(input("Enter the number of cells in the y-direction (ny): "))

In [ ]:
dx = float(input("Enter the cell size in the x-direction (dx, in degrees): "))
dy = float(input("Enter the cell size in the y-direction (dy, in degrees): "))

# 4. Grid generation
Click on the map to create the grid

In [ ]:
# Initialize the interactive map
try:
    grid_map
except NameError:
    grid_map = Map(center=(0.0, 0.0), zoom=2)


# Display the map
display(grid_map)


# Store polylines so they can be removed later
polylines = []
marker = []

# Display instructions
instructions = HTML(
    """
    <h4>Interactive Map for Grid Generation</h4>
    <ol>
        <li>Click anywhere on the map to find the origin coordinates (latitude and longitude).</li>
        <li>The grid will be visualized as a mesh directly on the map.</li>
    </ol>
    """
)
display(instructions)

# Function to handle clicks on the map
def handle_map_click(**kwargs):
    global marker, x0, y0
    
    if kwargs.get("type") == "click":  # Ensure this is a click event
        y0 = kwargs["coordinates"][0]
        x0 = kwargs["coordinates"][1]

        if marker in grid_map.layers:
            grid_map.remove_layer(marker)
            
        # Add a marker to the map at the clicked location
        marker = Marker(location=(y0, x0))
        grid_map.add_layer(marker)

        generate_grid(x0,y0)

def generate_grid(x0,y0):
    global polylines  # Access the global variable to update it
    global x_grid, y_grid 
        
    x_coords = np.linspace(x0, x0 + dx * nx, nx + 1)
    y_coords = np.linspace(y0, y0 + dy * ny, ny + 1)

    x_grid, y_grid = np.meshgrid(x_coords, y_coords)

     # Remove existing polylines from the map if they exist
    for polyline in polylines:
        if polyline in grid_map.layers:
            grid_map.remove_layer(polyline)
    polylines = []  # Clear the list after removing polylines

    # Collect all horizontal and vertical grid lines in one go
    all_grid_lines = []

    for j in range(ny):
        points = [(y_grid[j, i], x_grid[j, i]) for i in range(nx)]
        all_grid_lines.append(points)

    for i in range(nx):
        points = [(y_grid[j, i], x_grid[j, i]) for j in range(ny)]
        all_grid_lines.append(points)

    # Create a single Polyline layer for all grid lines
    polyline = Polyline(
        locations=all_grid_lines, 
        color="blue", 
        weight=1
    )
    
    # Add the grid lines to the map in one shot
    grid_map.add_layer(polyline)
    polylines.append(polyline)  # Keep track of this polyline
    
    #print("\nGrid generation complete!")
    #print(f"Grid dimensions: {nx} x {ny}")
    #print(f"x-coordinates range: {x_coords[0]} to {x_coords[-1]}")
    #print(f"y-coordinates range: {y_coords[0]} to {y_coords[-1]}")
    
# Attach the click handler to the map
grid_map.on_interaction(handle_map_click)


# 5. Save the grid to a MOHID-compatible file

In [ ]:
# Get current date and time
now = datetime.now()

# Format the date and time
formatted_date_time = now.strftime("%d-%m-%Y %H:%M:%S")

output_file = "mohid_grid.grd"
with open(output_file, "w") as f:
    f.write("PROJ4_STRING              : +proj=longlat +datum=WGS84 +no_defs\n")
    f.write("COMENT1                   : Grid generated by MOHID Jupyter Notebook\n")
    f.write("COMENT1                   : Generation Time: " + formatted_date_time + "\n")
    f.write("LATITUDE                  : " + str(y0) + "\n")
    f.write("LONGITUDE                 : " + str(x0) + "\n")
    f.write("COORD_TIP                 : 4\n")
    f.write("ILB_IUB                   : 1 " + str(ny) + "\n")
    f.write("JLB_JUB                   : 1 " + str(nx) + "\n")
    f.write("ORIGIN                    : " + str(x0) + " " + str(y0) + "\n")
    f.write("GRID_ANGLE                : 0\n")
    f.write("CONSTANT_SPACING_X        : 1\n")
    f.write("CONSTANT_SPACING_Y        : 1\n")
    f.write("DX                        : " + str(dx) + "\n")
    f.write("DY                        : " + str(dy) + "\n")

print(f"\nGrid saved to {output_file}")

# 6. Download or load GSHHG Coastline Data

In [ ]:
# Define URL and local paths
gshhg_url = "https://www.soest.hawaii.edu/pwessel/gshhg/gshhg-shp-2.3.7.zip"
zip_path = "gshhg_shapefiles.zip"
extract_path = "gshhg_data"

# Download the file if not exists
if not os.path.exists(zip_path):
    print("Downloading GSHHG data...")
    response = requests.get(gshhg_url)
    with open(zip_path, "wb") as f:
        f.write(response.content)

# Extract the files
if not os.path.exists(extract_path):
    print("Extracting GSHHG data...")
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_path)

print("GSHHG data is ready.")

# 7. Load and Filter Coastlines for Grid Region

In [ ]:
# Load the coastline shapefile
coastline_shapefile = os.path.join(extract_path, "GSHHS_shp", "f", "GSHHS_f_L1.shp") #if using downloaded GSHHG coastline data
#coastline_shapefile = "Coastlines/Para_3_completo.shp" #if using other coastline data

coastlines = gpd.read_file(coastline_shapefile)

# Define a bounding box
np_x = np.array(x_grid)
np_y = np.array(y_grid)

c = 3
x_min=np.min(np_x)-dx*c; y_min=np.min(np_y)-dy*c; x_max=np.max(np_x)+dx*c; y_max=np.max(np_y)+dy*c

bbox = [x_min, y_min, x_max, y_max] #[min_lon, min_lat, max_lon, max_lat]

# Create a bounding box geometry
bbox_geom = gpd.GeoDataFrame(geometry=[box(*bbox)], crs=coastlines.crs)

# Filter coastlines for the bounding box
filtered_coastlines = gpd.overlay(coastlines, bbox_geom, how='intersection')

# Convert to GeoJSON
coastline_geojson = filtered_coastlines.to_json()


# Convert to GeoJSON for ipyleaflet
#coastline_geojson = filtered_coastlines.__geo_interface__

# Create an interactive map centered in the region
center_lat = (bbox[1] + bbox[3]) / 2
center_lon = (bbox[0] + bbox[2]) / 2
m = Map(center=(center_lat, center_lon), zoom=8)

# Add coastlines
geo_layer = GeoData(geo_dataframe=filtered_coastlines, style={"color": "blue", "weight": 1})

# Add to map
m.add_layer(geo_layer)

# Display the map
m

# 8. Interpolate bathymetric data

In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter, label
from scipy.interpolate import griddata
from shapely.geometry import Point

# Remove NaN values from the dataset
mask = ~np.isnan(data['longitude']) & ~np.isnan(data['latitude']) & ~np.isnan(data['depth'])
lons, lats, depths = data['longitude'][mask], data['latitude'][mask], data['depth'][mask]

# Initialize the grid with -99
zi = np.full((x_grid.shape[0] - 1, x_grid.shape[1] - 1), -99, dtype=float)

# Convert coastline geometries to Shapely Polygons
coast_polygons = list(filtered_coastlines.geometry)

# Check if a point is inside any coastline polygon
def is_water(x, y):
    return not any(poly.contains(Point(x, y)) for poly in coast_polygons)

# Generate water mask with the correct shape
water_mask = np.vectorize(is_water)(x_grid[:-1, :-1], y_grid[:-1, :-1])

# Interpolate depth values only for water cells
zi[water_mask] = griddata(
    (lons, lats), depths,
    (x_grid[:-1, :-1][water_mask], y_grid[:-1, :-1][water_mask]),
    method="nearest"
)

# Apply depth constraints
zi[zi < 0] = 0  # Ensure minimum depth is 0

# Smooth the valid water data using Gaussian filter
zi = gaussian_filter(zi * water_mask, sigma=1)

# Restore land cells to -99
zi[~water_mask] = -99

# Identify and remove small isolated water patches
labeled_array, num_features = label(water_mask)
for label_idx in range(1, num_features + 1):
    component_mask = labeled_array == label_idx
    if np.sum(component_mask) < 20:  # Threshold for isolated cells
        zi[component_mask] = -99

# 9. Load a previously generated Mohid griddata file

In [ ]:
# Load grid data from file
file_path = "Batim_IlhasAcores_Level2_3km.dat"
#file_path = "mohid_griddata.dat"
# Load a MOHID grid data file
with open(file_path, 'r') as f:
    lines = f.readlines()

grid_data = []
x_coords = []
y_coords = []
n_rows, n_cols = None, None  # Use None to detect missing values
start_reading_xx = False
start_reading_yy = False
start_reading_grid = False

for line in lines:
    line = line.strip()  # Remove leading and trailing spaces
    parts = line.split()

    if line.startswith("ILB_IUB"):
        n_rows = int(parts[3]) 
    elif line.startswith("JLB_JUB"):
        n_cols = int(parts[3])
    elif line.startswith("ORIGIN"):
        x0 = float(parts[2])
        y0 = float(parts[3])
    elif line.startswith("DX"):
        dx = float(parts[2])
    elif line.startswith("DY"):
        dy = float(parts[2])
    elif "<BeginXX>" in line:
        start_reading_xx = True
        continue
    elif "<EndXX>" in line:
        start_reading_xx = False
    elif start_reading_xx:
        try:
            x_coords.append(float(line))
        except ValueError:
            print(f"Warning: Skipping invalid line -> {line}")
    elif "<BeginYY>" in line:
        start_reading_yy = True
        continue
    elif "<EndYY>" in line:
        start_reading_yy = False
    elif start_reading_yy:
        try:
            y_coords.append(float(line))
        except ValueError:
            print(f"Warning: Skipping invalid line -> {line}")
    elif "<BeginGridData2D>" in line:
        start_reading_grid = True
        continue
    elif "<EndGridData2D>" in line:
        start_reading_grid = False
    elif start_reading_grid:
        try:
            grid_data.append(float(line))
        except ValueError:
            print(f"Warning: Skipping invalid line -> {line}")

# Debugging prints
print(f"Extracted Dimensions: n_rows={n_rows}, n_cols={n_cols}")
print(f"Grid Data Length: {len(grid_data)}")

if not x_coords:
    x_coords = np.linspace(x0, x0 + dx * n_cols, n_cols+1)
    y_coords = np.linspace(y0, y0 + dy * n_rows, n_rows+1)
else:
    x_coords = np.array(x_coords) + x0
    y_coords = np.array(y_coords) + y0
    
# Ensure grid dimensions are valid
if n_rows is None or n_cols is None:
    raise ValueError("Grid dimensions could not be determined from the file.")

# Check if data size matches expected shape
expected_size = n_rows * n_cols
if len(grid_data) != expected_size:
    raise ValueError(f"Mismatch: Grid data size {len(grid_data)} does not match expected ({expected_size}).")

# Convert to NumPy array and reshape correctly
zi = np.array(grid_data).reshape(n_rows, n_cols)

x_grid, y_grid = np.meshgrid(x_coords, y_coords)
    
print(f"Loaded grid data shape: {zi.shape}")

# 10. Visualize and update depth values by clicking on the map
This code is currently efficient for not-so-large griddatas (e.g., 100 x 100 cells). To update large gridatas (e.g., 400 x 400 cells), convert first to shapefile, modify the cell depths in QGIS, and then convert the griddata to a MOHID-compatible file again. You can use this Jupyter Notebook to convert between formats.

In [ ]:
# Grid setup
n_rows, n_cols = zi.shape
np_x, np_y = np.array(x_grid), np.array(y_grid)
min_lon, max_lon = np_x.min(), np_x.max()
min_lat, max_lat = np_y.min(), np_y.max()

# Create the map
m = Map(center=(np_y.mean(), np_x.mean()), zoom=8)
marker, grid_layer = None, None

def get_color(value):
    """Convert depth value to a color using 'viridis' colormap."""
    if value == -99: return "#ffffff00"  # Transparent for invalid values
    norm = Normalize(vmin=zi[zi != -99].min(), vmax=zi.max())
    return mcolors.to_hex(plt.cm.viridis(norm(value))[:3])

def generate_grid_geojson():
    """Create a GeoJSON grid representation."""
    features = [
        {
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [[
                    [min_lon + (max_lon - min_lon) * j / (n_cols - 1), min_lat + (max_lat - min_lat) * i / (n_rows - 1)]
                    for j, i in [(j, i), (j + 1, i), (j + 1, i + 1), (j, i + 1), (j, i)]
                ]]
            },
            "properties": {
                "fill": get_color(zi[i, j]),
                "stroke": "#000000", "fill-opacity": 0.6, "stroke-width": 0.5,
                "i": i, "j": j
            }
        }
        for i in range(n_rows) for j in range(n_cols) if zi[i, j] != -99
    ]
    return {"type": "FeatureCollection", "features": features}

def update_map():
    """Update the GeoJSON grid layer on the map."""
    global grid_layer
    if grid_layer: m.remove_layer(grid_layer)  # Remove old layer
    grid_layer = GeoJSON(
        data=generate_grid_geojson(),
        style_callback=lambda f: {
            "fillColor": f["properties"]["fill"],
            "color": f["properties"]["stroke"],
            "weight": f["properties"]["stroke-width"],
            "fillOpacity": f["properties"]["fill-opacity"]
        }
    )
    m.add_layer(grid_layer)

def get_grid_index(lon, lat):
    """Convert lat/lon to grid indices."""
    j, i = int((lon - min_lon) / (max_lon - min_lon) * (n_cols - 1)), int((lat - min_lat) / (max_lat - min_lat) * (n_rows - 1))
    return (j, i) if 0 <= j < n_cols and 0 <= i < n_rows else (None, None)

def update_depth(**kwargs):
    """Handle map click, update depth, and refresh grid."""
    global marker
    if kwargs.get("type") != "click": return

    lat, lon = kwargs["coordinates"]
    if marker: m.remove_layer(marker)  # Remove previous marker
    marker = Marker(location=(lat, lon))
    m.add_layer(marker)

    j, i = get_grid_index(lon, lat)
    if j is None or i is None: return

    input_box = widgets.Text(
        placeholder='Enter new depth value...',
        description=f'Cell ({i}, {j}) Depth {zi[i, j]:.1f}:',
        disabled=False,
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px')
    )

    submit_button = widgets.Button(description="Submit")

    def on_submit(_):
        try:
            zi[i, j] = float(input_box.value)
            update_map()  # Refresh entire grid
        except ValueError:
            print("Invalid input. Enter a numeric value.")

    submit_button.on_click(on_submit)
    clear_output(wait=True)  # Clear previous outputs
    display(m)  # Re-display the map
    display(input_box, submit_button)  # Show input box and submit button

update_map()
display(m)
m.on_interaction(update_depth)


# 11. Save the griddata to a MOHID-compatible file

In [ ]:
rows, cols = zi.shape

np_x = np.array(x_grid)
np_y = np.array(y_grid)

# Calculate the cell spacing
dx = np.abs(np_x[0][1] - np_x[0][0])
dy = np.abs(np_y[1][0] - np_y[0][0])

# Get current date and time
now = datetime.now()

# Format the date and time
formatted_date_time = now.strftime("%d-%m-%Y %H:%M:%S")

output_file = "mohid_griddata.dat"
with open(output_file, "w") as f:
    f.write("PROJ4_STRING              : +proj=longlat +datum=WGS84 +no_defs\n")
    f.write("COMENT1                   : Grid generated by MOHID Jupyter Notebook\n")
    f.write("COMENT1                   : Generation Time: " + formatted_date_time + "\n")
    f.write("LATITUDE                  : " + str(y0) + "\n")
    f.write("LONGITUDE                 : " + str(x0) + "\n")
    f.write("COORD_TIP                 : 4\n")
    f.write("ILB_IUB                   : 1 " + str(int(rows)) + "\n")
    f.write("JLB_JUB                   : 1 " + str(int(cols)) + "\n")
    f.write("ORIGIN                    : " + str(x0) + " " + str(y0) + "\n")
    f.write("GRID_ANGLE                : 0\n")
    f.write("CONSTANT_SPACING_X        : 1\n")
    f.write("CONSTANT_SPACING_Y        : 1\n")
    f.write("DX                        : " + str(dx) + "\n")
    f.write("DY                        : " + str(dy) + "\n")
    f.write("FILL_VALUE                : -99\n")
    f.write("<BeginGridData2D>\n")
    
    for i in range(rows):
        for j in range(cols):
            f.write(f"{zi[i][j]:.1f}\n")
            
    f.write("<EndGridData2D>")       
    

print(f"\nGrid saved to {output_file}")

# 12. Convert the griddata to a shapefile

In [ ]:
# Create a shapefile writer object
w = shapefile.Writer('depth_grid_cells', shapefile.POLYGON)
w.field('i', 'N')       # Row index (i)
w.field('j', 'N')       # Column index (j)
w.field('Depth', 'F', decimal=2)  # Depth value (float)

# Loop through the grid and create polygon cells
for i in range(len(zi)):
    for j in range(len(zi[i])):
        depth = zi[i, j]  # Get depth value
                
        # Define the 4 corner points of the grid cell with double precision
        lon1, lat1 = round(x_grid[i, j], 15), round(y_grid[i, j], 15)         # Bottom-left
        lon2, lat2 = round(x_grid[i, j+1], 15), round(y_grid[i, j+1], 15)     # Bottom-right
        lon3, lat3 = round(x_grid[i+1, j+1], 15), round(y_grid[i+1, j+1], 15) # Top-right
        lon4, lat4 = round(x_grid[i+1, j], 15), round(y_grid[i+1, j], 15) # Top-left

        # Create a polygon for the grid cell
        w.poly([[ (lon1, lat1), (lon2, lat2), (lon3, lat3), (lon4, lat4), (lon1, lat1) ]])  
        
        # Add attributes (Row, Col, Depth)
        w.record(i+1, j+1, depth)

# Save the shapefile
w.close()

print("Shapefile 'depth_grid_cells.shp' created successfully.")


# 13. Save shapefile to MOHID griddata

In [ ]:
# Load the shapefile
shapefile_path = 'depth_grid_cells.shp'
gdf = gpd.read_file(shapefile_path)

# Get the bounds of the shapefile
bounds = gdf.total_bounds  # [minx, miny, maxx, maxy]
minx, miny, maxx, maxy = bounds
origin_coordinates = (minx, miny)

def calculate_spacing(gdf):
    # Ensure the GeoDataFrame has geometries
    if gdf.empty or gdf.geometry.iloc[0] is None:
        raise ValueError("GeoDataFrame is empty or has no valid geometries.")

    # Get the first polygon's coordinates
    first_polygon = gdf.geometry.iloc[0]

    # Validate that the geometry is a valid polygon
    if not first_polygon.is_valid or first_polygon.geom_type != 'Polygon':
        raise ValueError("The first geometry is not a valid Polygon.")

    # Extract unique x and y coordinates
    x_coords = sorted(set(pt[0] for pt in first_polygon.exterior.coords))
    y_coords = sorted(set(pt[1] for pt in first_polygon.exterior.coords))

    # Ensure there are enough points to calculate spacing
    if len(x_coords) < 2 or len(y_coords) < 2:
        raise ValueError("Not enough unique coordinates to calculate spacing.")

    # Calculate spacing between unique coordinates
    dx = x_coords[1] - x_coords[0]  # Difference between the first two unique x-coordinates
    dy = y_coords[1] - y_coords[0]  # Difference between the first two unique y-coordinates

    return dx, dy


# Get cell spacing
dx, dy = calculate_spacing(gdf)
print(f"Cell spacing - dx: {dx}, dy: {dy}")

rows = int(gdf['i'].max()+1)
cols = int(gdf['j'].max()+1)

# Get current date and time
now = datetime.now()

# Format the date and time
formatted_date_time = now.strftime("%d-%m-%Y %H:%M:%S")

output_file = "mohid_griddata_from_shapefile.dat"
with open(output_file, "w") as f:
    f.write("PROJ4_STRING              : +proj=longlat +datum=WGS84 +no_defs\n")
    f.write("COMMENT1                  : Grid generated by MOHID Jupyter Notebook\n")
    f.write("COMMENT2                  : Generation Time: " + formatted_date_time + "\n")
    f.write("LATITUDE                  : " + str(miny) + "\n")
    f.write("LONGITUDE                 : " + str(minx) + "\n")
    f.write("COORD_TIP                 : 4\n")
    f.write("ILB_IUB                   : 1 " + str(rows) + "\n")
    f.write("JLB_JUB                   : 1 " + str(cols) + "\n")
    f.write("ORIGIN                    : " + str(minx) + " " + str(miny) + "\n")
    f.write("GRID_ANGLE                : 0\n")
    f.write("CONSTANT_SPACING_X        : 1\n")
    f.write("CONSTANT_SPACING_Y        : 1\n")
    f.write("DX                        : " + str(dx) + "\n")
    f.write("DY                        : " + str(dy) + "\n")
    f.write("FILL_VALUE                : -99\n")
    f.write("<BeginGridData2D>\n")

    zi = np.full((rows, cols), -99.0)  # Fill with -99 as the default value

    for index, row in gdf.iterrows():
        i, j, depth = int(row['i']), int(row['j']), row['Depth']
        zi[i-1, j-1] = depth
    
    for i in range(rows):
        for j in range(cols):
            f.write(f"{zi[i][j]:.1f}\n")

    f.write("<EndGridData2D>\n")

print(f"\nGrid saved to {output_file}")

# 14. Plot MOHID griddata

In [ ]:
np_x = np.array(x_grid)
np_y = np.array(y_grid)

x_min = np.min(np_x)
y_min = np.min(np_y)
x_max = np.max(np_x)
y_max = np.max(np_y)

#extent = [x_max, x_min, y_max, y_min]

# Calculate the cell spacing
dx = np.abs(np_x[0][1] - np_x[0][0])
dy = np.abs(np_y[1][0] - np_y[0][0])

# Expand the extent by n cells in all directions
n = 5

lon_min = x_min - n * dx
lon_max = x_max + n * dx
lat_min = y_min - n * dy
lat_max = y_max + n * dy

extent = [
    lon_min,  # Left
    lon_max,  # Right
    lat_min,  # Bottom
    lat_max]  # Top

def calculate_zoom_level():
    """
    Calculate zoom level based on the geographic extent.
    """
    # Approximate calculation for zoom level based on extent
    lat_range = lat_max - lat_min
    lon_range = lon_max - lon_min

    # Smaller ranges mean higher zoom levels
    range_avg = max(lat_range, lon_range)  # Focus on the larger dimension
    zoom = int(np.log2(360 / range_avg))   # Base-2 logarithm for zoom estimation

    # Limit zoom levels to reasonable values (e.g., 1 to 19)
    return max(1, min(zoom, 19))  

# Calculate zoom level
zoom_level = calculate_zoom_level()
print(f"Automatically calculated zoom level: {zoom_level}")

# Set the image size and create the figure
fig = plt.figure(figsize=(15, 15))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent(extent)

def image_spoof(self, tile):
    url = self._image_url(tile)  # Get the URL of the street map API
    req = Request(url)  # Start request
    req.add_header('User-agent', 'Anaconda 3')  # Add user agent to request
    fh = urlopen(req)
    im_data = io.BytesIO(fh.read())  # Get image
    fh.close()  # Close URL
    img = Image.open(im_data)  # Open image with PIL
    img = img.convert(self.desired_tile_form)  # Set image format
    return img, self.tileextent(tile), 'lower'  # Reformat for cartopy

cimgt.GoogleTiles.get_image = image_spoof  # Reformat web request for street map spoofing
osm_img = cimgt.GoogleTiles(style='satellite')
ax.add_image(osm_img, zoom_level) #you can increase the zoom level for better resolution of the satellite image

zi_mask = np.ma.masked_array(zi, zi <= -99)  # Mask to set transparency to a certain value

# Normalizing the data for the colorbar
norm = Normalize(vmin=np.min(zi_mask), vmax=np.max(zi_mask))

# Plot pcolormesh
pc = ax.pcolormesh(x_grid, y_grid, zi_mask, cmap='viridis', norm=norm)

# Plot scatter data if available
if 'depths' in locals() and 'lons' in locals() and 'lats' in locals():
    scatter = ax.scatter(lons, lats, c=depths, s=0.1, cmap='viridis', norm=norm, label='Data Points')

#Title
plt.title('Interpolated Bathymetric Data', fontsize=18, loc='center')

# Adjust the size of the colorbar
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1, axes_class=plt.Axes)

cbar = plt.colorbar(pc, cax=cax, orientation="vertical")
cbar.set_label('Depth (m)', labelpad=25, rotation=270, fontsize=16)
cbar.ax.tick_params(labelsize=14)

plt.savefig(rf'Griddata.png', format='png', dpi=300, bbox_inches='tight')